# Computational Lab 3

# Simulating polymers as random walks

## Learning Objectives

 - Apply Monte Carlo methods to describe polymers. 
 - Understand random walk algorithms and how they can be used to simulate polymers.
 - Understand the relationship between polymer length and end-to-end distance.
 - Use python and `matplotlib` to analyse and plot quantities related to random walk polymers.
 - Understand the key differences between random walks/polymers and self-avoiding walks/polymers.


This notebook was adapted from various sources:

- [The SciPython Book example on random flight polymers](https://scipython.com/book/chapter-4-the-core-python-language-ii/examples/the-random-flight-polymer/)
- [Anton Akhmerov's Delft University of Technology lectures](https://compphys.quantumtinkerer.tudelft.nl)


Authors:
- Micaela Matta

## Background: polymers and how to characterize them

Polymers are large molecules composed of multiple repeating subunits (monomers). They are essential to our everyday lives, from biological processes to synthetic materials (see Polymers on Wikipedia for more information).
We aim to simulate linear, unbranched polymers in a good solvent, where the polymers are separated from each other and do not interact. This allows us to consider the physics of a single polymer in isolation.

One particular property of a polymer is its lack of rigidity. Generally, polymers are quite floppy. We won't describe this from a microscopic model, such as quantum chemistry or molecular dynamics (see the rest of this module). Instead, we'll use a simpler toy model. Before we do that, let's consider how we can quantify a polymer.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/Ideal_chain_random_walk.svg/2560px-Ideal_chain_random_walk.svg.png" width=300>

The image above shows a linear polymer consisting of $N$ subunits is then described by a ordered set of points $\{\mathbf{r}_0, \mathbf{r}_1, \dots, \}$. Note that there are $N$ subunits, but $N-1$ bonds between the subunits. 

We can characterize the polymer by its end-to-end distance $R$:

$$R = \left|\mathbf{r}_N - \mathbf{r}_0\right|\,.$$

Alternatively, we can use the radius of gyration squared that is given by


$$R_g^2 = \frac{1}{N} \sum_{i=0}^N \left|\mathbf{r}_i - \mathbf{r}_\text{cm}\right|^2\,,$$

where $$\mathbf{r}_\text{cm} = \frac{1}{N+1} \sum_{i=0}^N \mathbf{r}_i$$ 

is the center of mass of the polymer. 

The radius of gyration describes how "curled up" the polymer is.

## Simulating Polymers with Monte Carlo

The Monte Carlo algorithm is a widely used approach for simulating the behavior of a polymer. It is based on a random walk algorithm, where a random step is taken to represent the motion of a single monomer. In this way, the behavior of a polymer can be simulated by a series of random steps. Python provides a powerful set of libraries for implementing Monte Carlo algorithms, making it an ideal language for simulating polymers.


## Imports and everything

As always, let's start with importing the necessary packages.

In [ ]:
# run this cell before any of the cells below, 
# or you'll get a NameError for trying to use a module that's not been imported

import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import numpy as np
import random
import math
%matplotlib inline

# Generating a Random Walk Polymer in 2 Dimensions in Python

In this section, we will use Python to generate a random walk in two dimensions, and calculate the radius of gyration and end-to-end distances of the polymer.

## Initializing the Polymer

We will begin by importing the necessary libraries and setting a random seed. This will enable us to reproduce the same results each time the code is run.

In [ ]:
# Set the random seed
np.random.seed(42)

Next, we will set the number of monomers in the polymer and initialize the polymer array.

In [ ]:
# Set the number of monomers
N_monomers = 1000
# Initialize the polymer position vectors 
positions = np.zeros((N_monomers,2))

## Generating the Polymer

We will now loop over the monomers and generate a random position for each.

In [ ]:
# Loop over the monomers and generate a random position for each
cx , cy = 0, 0
for i in range(1, N_monomers):
    # Pick a random orientation for the next segment.
    theta = random.random() * 2. * np.pi
    # Add on the corresponding displacement vector for this segment.
    positions[i, 0]  = positions[i-1, 0] + np.cos(theta)
    positions[i, 1]  = positions[i-1, 1] + np.sin(theta)
    # Store it, and update or centre of mass sum.
    cx, cy = cx + positions[i, 0], cy + positions[i, 1]
    
# Calculate the position of the centre of mass.
cx, cy = cx / N_monomers, cy /N_monomers

# Finally, re-centre our polymer on the centre of mass.
for i in range(N_monomers):
    positions[i,:] = positions[i,0]-cx, positions[i,1]-cy

## Plotting the Polymer

We can now plot the polymer chain.

In [ ]:
# Plot the resulting polymer chain
plt.plot(positions[:, 0], positions[:, 1], '-', label= "2D random walk")
plt.scatter(positions[0, 0], positions[0, 1], label="Start")
plt.scatter(positions[-1, 0], positions[-1, 1], label="End")
# labels and title
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.title(f'Random Flight Polymer with {N_monomers} monomers')
plt.show()

## Calculating the Radius of Gyration and End-to-End Distance

The radius of gyration $R_g$ describes how "curled up" the polymer is.
We can now calculate it by applying the equation:

$$R_g = \sqrt{\frac{1}{N}\sum_{i=1}^N\left(r_i-r_0\right)^2}$$

where $r_i$ are the coordinates of the $i^{th}$ monomer and $r_0$ are the coordinates of the first monomer.

In [ ]:
# The polymer segment positions are already given relative to the centre of 
# mass, so this is just the rms position of the segments.
rg = np.sqrt(np.mean(np.sum(positions**2, axis=1)))

To calculate the end-to-end distance, we consider the vector between the first and last monomer:


$$r_{ee} = |r_{N} - r_{0}|$$

In [ ]:
# Calculate the end-to-end distance
end_to_end_distance = np.linalg.norm(positions[0, :] - positions[N_monomers-1, :] )

In [ ]:
print("Radius of gyration =", rg)
print("End-to-end distance =", end_to_end_distance)

<div class="alert alert-success"><b>Check your understanding: </b>
    
Can you generalise the code example above for a 3-dimensional polymer?
    
</div>

# 3D random walks

We are now switching to 3-dimensional random walks. We will adapt the code seen above to calculate x/y/z positions of a random walk chain.



## Picking the orientation of the next segment

One way to pick the location of the next segment is to sample a point on the unit sphere using spherical polar coordinates. This can be done by generating two random numbers, one in the range [0,2π] and the other in the range [0,π], then using the corresponding pair of angles in the spherical polar coordinate system to calculate the x, y, and z coordinates of the point. 

An example of this method is shown in the code below:

```
theta = np.arccos(2 * random.random() - 1)
phi = random.random() * 2. * np.pi
positions[i, 0]  = positions[i-1, 0] + np.cos(theta) * np.sin(phi)
positions[i, 1]  = positions[i-1, 1] + np.sin(theta) * np.sin(phi)
positions[i, 2]  = positions[i-1, 2] + np.cos(phi)
```


An alternative to this approach is to enumerate the possible moves on a grid - we will see this approach implemented in the section of this notebook describing self-avoiding polymers.

## `generate_random_walk_3D`

Below is a function that generates a random walk in 3D and returns the positions of each monomer, the end-to-end distance and radius of gyration.

In [ ]:
def generate_random_walk_3D(N_monomers):
    """Generates a random walk with N_monomers steps in 3D, returning the positions of each monomer, the end-to-end distance and the radius of gyration.

    Parameters
    ----------
    N_monomers: int
        Number of monomers in the random walk.

    Returns
    -------
    positions: ndarray
        An N_monomers x 3 array of positions of the monomers.
    end_to_end_distance: float
        The end-to-end distance of the random walk.
    rg: float
        The radius of gyration of the random walk.
    """
    # initialize the polymer array
    positions = np.zeros((N_monomers, 3))
    # Set the first position to the origin
    positions[0,:] = (0,0,0)

    # Sum of positions
    cx, cy, cz = 0, 0, 0

    # Loop over the monomers and generate a random position for each
    for i in range(1, N_monomers):
        # Pick a random orientation for the next segment.
        theta = np.random.uniform(0, 2*np.pi)
        phi = np.random.uniform(0, np.pi)
        # Add on the corresponding displacement vector for this segment.
        positions[i, 0]  = positions[i-1, 0] + np.cos(theta) * np.sin(phi)
        positions[i, 1]  = positions[i-1, 1] + np.sin(theta) * np.sin(phi)
        positions[i, 2]  = positions[i-1, 2] + np.cos(phi)
        # Store it, and update or centre of mass sum.
        cx, cy, cz = cx + positions[i, 0], cy + positions[i, 1], cz + positions[i, 2]

    # Calculate the position of the centre of mass.
    cx, cy, cz = cx / N_monomers, cy /N_monomers, cz / N_monomers

    # Re-centre our polymer on the centre of mass.
    positions -= np.array([cx, cy, cz])

    # Calculate the end-to-end distance
    end_to_end_distance = np.linalg.norm(positions[0, :] - positions[N_monomers-1, :] )

    # Calculate the radius of gyration
    rg = np.sqrt(np.mean(np.sum(positions**2, axis=1)))

    return positions, end_to_end_distance, rg


Let's run the function once:

In [ ]:
# Generate 1 random walk 
N_monomers = 100
positions, e2e, rg = generate_random_walk_3D(N_monomers)

And now we can do our first 3D plot!!

In [ ]:
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

# Plot the coordinates of the random walk
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

c1, c2 =sns.palettes.color_palette()[1:3] # set the colors
# plot 
ax.plot(positions[:, 0], positions[:, 1], positions[:, 2], label="3D Random Walk")
ax.scatter(positions[0, 0], positions[0, 1], positions[0, 2], color=c1 ,label="Start")
ax.scatter(positions[-1, 0], positions[-1, 1], positions[-1, 2], color=c2, label="End")
# titles, legend
ax.set_title(f"3D Random Walk of length {N_monomers}")
ax.legend()
plt.show()


<div class="alert alert-success"><b>Check your understanding: </b>
    
Run the function again, this time changing parameters. 

What happens to the radius of gyration and end-to-end distance if you use different `N_monomers`?
    
</div>

In [ ]:
e2e

In [ ]:
rg

# Sampling Many Polymer Chains


## Statistics over Multiple Walks

In order to generate statistically significant data, we'd like to sample the behaviour of many polymer chains just like the one above. 

To do so, we can call multiple times the function that generates a single random-walk-polymer and calculate the average properties over the whole population.


In [ ]:
# Generate random polymers
N_polymers = 1000
N_monomers = 200

# Initialize arrays to store the end-to-end distance and radius of gyration
end_to_end_distances = np.zeros(N_polymers)
rgs = np.zeros(N_polymers)

# Generate the polymers
for i in range(N_polymers):
    _, end_to_end_distance, rg, = generate_random_walk_3D(N_monomers)
    # Save the end to end distances
    end_to_end_distances[i] = end_to_end_distance
    rgs[i] = rg

## Probability Density Function

We can now compare the calculated end-to-end distance to the theoretically-predicted probability density functions. 


The theoretical probability density function for the end-to-end distance of a 3-dimensional random walk polymer is

$$P(R_{ee}) =  4\pi R_{ee}^2 \left(\frac{2\pi N}{3}\right)^{3/2} exp\left({-\frac{3R_{ee}^2}{2N}}\right)$$


where $R_{ee}$ is the end-to-end distance and $N$ is the number of monomers.

In [ ]:
# Plot the histogram
plt.hist(end_to_end_distances, bins=50, label="End-to-end Distance")

plt.title(f"{N_polymers} Random Walk Polymers of length {N_monomers} in 3D")
plt.xlabel("Distance")
plt.ylabel("Frequency")
plt.legend()
plt.show()

In order to compare with probability density functions, we need to normalise the histogram by changing the argument `density`

In [ ]:
# Plot normalised histogram
plt.hist(end_to_end_distances, bins=50, density=True, label=r'End-to-end distances')

# compare with probability density function
r = np.linspace(0,40,1000)

Pr = 4.*np.pi*r**2 * (2 * np.pi * N_monomers / 3)**-1.5 * np.exp(-3*r**2 / 2 / N_monomers)
plt.plot(r, Pr, lw=4, label=r'Probability Density Function')
plt.xlabel("Distance")
plt.ylabel("Frequency")
plt.legend()
plt.show()

<div class="alert alert-success"><b> Check your understanding: </b>

- What happens if you increase or decrease the parameter `N_monomers`, the polymer length?
- What happens if you increase or decrease the parameter `N_polymers`, the number of random flight polymers?


##  End-to-End Distance and Polymer Length


We now want to see what happens to the end-to-end distance if we start varying the polymer length.
Below, we generate a series of random walk polymers of different length (`N_monomers`). For each, we calculate the average end to end distance $R_{ee}$ over a number of iterations (`N_polymers`).

In [ ]:
# Generate random polymers with different lengths
N_polymers = 10
lengths = [10, 100, 500, 1000, 5000, 10000]

# Initialize arrays to store the end-to-end distance and radius of gyration
end_to_end_distances = np.zeros((len(lengths), N_polymers))
rgs = np.zeros((len(lengths), N_polymers))

# Generate the polymers
for i, length in enumerate(lengths):
    for j in range(N_polymers):
        _, end_to_end_distance, rg = generate_random_walk_3D(length)
        # Store end to end distance
        end_to_end_distances[i, j] = end_to_end_distance
        rgs[i, j] = rg

# Calculate the average end-to-end distance for each polymer length
average_end_to_end_distances = np.mean(end_to_end_distances, axis=1)
average_rg = np.mean(rgs, axis=1)

In [ ]:
average_end_to_end_distances

In [ ]:
average_rg

We now plot the results:

In [ ]:
# Plot the results
plt.plot(lengths, average_end_to_end_distances,  'o-', label="End-to-end distance")
plt.plot(lengths, average_rg,  'o-', label="Radius of gyration")
plt.title(f"Average over {N_polymers} polymers")
plt.legend()
plt.show()

<div class="alert alert-success"><b>Check your understanding: </b>
    
What happens to the plot above if you increase `N_polymers`, that is, the number of random walks used to calculate `average_end_to_end_distances` and `average_rg` ? 
</div>

We can observe that the end-to-end distance and radius of gyration increase with the increase in polymer length (`N_monomers`), following a trend described in lectures 5-6.

<div class="alert alert-success"><b> Task: 2.1 </b>
Scaling Laws  

- Find the scaling law for the end-to-end distance and radius of gyration of a 3D random walk polymer as a function of the polymer length.
- Plot the average end to end distance and end-to-end distance together with the corresponding analytical functions describing their scaling behaviour. Upload your plot(s) to KEATS. 
</div>

# Self-avoiding polymer chains

In  code examples in this notebook, we have not restricted the positions of each monomer with respect to the previous ones. This can lead to some monomers overlapping in space and unphysical results: the random walk model is in fact an oversimplification of a real polymer chain.

<img src="https://gitlab.kwant-project.org/computational_physics/lectures/-/raw/master/src/figures/polymer-selfavoiding.svg" width="200"/>


Self-avoidance is a complex example of long-range correlations, where the next step of the walk depends on the entire history of the walk. Most of our knowledge about self-avoiding walks comes from computer simulations. 

The example below describes a self avoiding polymer on a square grid. This representation follows a similar principle as the game of [Snake](https://en.wikipedia.org/wiki/Snake_(video_game_genre) and is quite common in Monte Carlo implementations of self-avoiding polymers, as it allows to reduce the number of "allowed" moves.


In [ ]:
def get_possible_directions(point):
    """
    Returns a list of possible directions for the given point.

    Parameters
    ----------
    point : tuple
        The point in the form (x, y, z).

    Returns
    -------
    list
        A list of coordinates in the form of (x, y, z) that are the possible directions of the given point.
    """
    x, y, z = point
    directions = [
    (x+1, y, z),  # right
    (x-1, y, z),  # left
    (x, y+1, z),  # forward
    (x, y-1, z),  # backward
    (x, y, z+1),  # up
    (x, y, z-1)   # down
    ]
    return directions

def self_avoiding_walk_3D(N):
    """
    Generates a 3D self-avoiding random walk with N steps.

    Parameters
    ----------
    N : int
        The number of steps in the self-avoiding random walk.

    Returns
    -------
    positions : array of positions in the self-avoiding random walk.
    """

    Nsteps = range(N)
    current_position = np.array([0, 0, 0])
    visited_points = []
    for _ in Nsteps:
        visited_points.append(current_position)
        all_directions = get_possible_directions(current_position)
        not_visited_directions = [direction for direction in all_directions if np.all(direction != visited_points)]
        current_position = random.choice(not_visited_directions)

    positions = np.array(visited_points)
    return positions


In [ ]:
N= 1000
positions = self_avoiding_walk_3D(N)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
# Plot the coordinates of the random walk
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

c1, c2 =sns.palettes.color_palette()[1:3] 
ax.plot(positions[:,0],positions[:,1],positions[:,2],label="Self-avoiding Walk")
ax.scatter(positions[0,0],positions[0,1],positions[0,2], color=c1 ,label="Start")
ax.scatter(positions[-1,0],positions[-1,1],positions[-1,2], color=c2, label="End")
ax.set_title(f"3D Self-avoiding walk of length {N}")
ax.legend()
plt.show()

<div class="alert alert-success"><b> Task 2.2: </b>
Self avoiding walk algorithm

Write a pseudocode (in the form of a bullet point list) to generate a self-avoiding walk.
    
</div>

<div class="alert alert-success"><b> Task 2.3: </b>
Self avoiding walk function


Comment each line of the function `self_avoiding_walk_3D`.
    
</div>

<div class="alert alert-block alert-info">
<b> Key points:</b>
    
- Polymer behaviour is complex; here we have adopted some very crude approximations to describe "ideal" polymers
- Random walks are suitable to describe brownian motion, ideal polymer chains, and other important physical/chemical phenomena.
    
- Monte Carlo simulations can provide a computational shortcut to describe complex phenomena where analytical solutions are hard (or not known)
    
    
</div>

<img src="https://i.ytimg.com/vi/P4LhWSN3YSw/maxresdefault.jpg" width=600>